# Обработка данных для функциональности "Определение обсценной лексики"

##  Загрузки

### Загрузка корпуса с обсценной лексикой

In [ ]:
!wget https://raw.githubusercontent.com/odaykhovskaya/obscene_words_ru/master/obscene_corpus.txt

--2021-02-24 09:48:11--  https://raw.githubusercontent.com/odaykhovskaya/obscene_words_ru/master/obscene_corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152344 (149K) [text/plain]
Saving to: ‘obscene_corpus.txt.1’

obscene_corpus.txt. 100%[===================>] 148.77K  --.-KB/s    in 0.03s   

2021-02-24 09:48:11 (5.46 MB/s) - ‘obscene_corpus.txt.1’ saved [152344/152344]



### Загрузка корпуса с 2ch

In [ ]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/2ch_corpus.txt.zip

--2021-02-24 09:35:45--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/2ch_corpus.txt.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/2ch_corpus.txt.zip [following]
--2021-02-24 09:35:45--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/2ch_corpus.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6255552 (6.0M) [application/zip]
Saving to: ‘2ch_corpus.txt.zip’

2ch_corpus.txt.zip  100%[===================>]   5.96M  --.-KB/s    in 0.1s    

2021-02-24 09:35:47 (56.5 MB/s) - ‘2ch_cor

In [ ]:
!unzip 2ch_corpus.txt.zip

Archive:  2ch_corpus.txt.zip
  inflating: 2ch_corpus.txt          
  inflating: __MACOSX/._2ch_corpus.txt  


### Импорты и пути

In [10]:
import sys
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
from string import punctuation

tokenizer = nltk.tokenize.WhitespaceTokenizer()
morph = MorphAnalyzer()

punct = punctuation+'«»—…“”*№–'
sys.path.append('../')
sys.path.append('./')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


In [9]:
sys.path

['/Users/vladislavasan/PycharmProjects/ugc_analyzer/notebooks/preprocessing',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python38.zip',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Users/vladislavasan/PycharmProjects/ugc_analyzer/venv/lib/python3.8/site-packages',
 '/Users/vladislavasan/PycharmProjects/ugc_analyzer/venv/lib/python3.8/site-packages/IPython/extensions',
 '/Users/vladislavasan/.ipython',
 '../']

In [13]:
obscene_data_path = "/Users/vladislavasan/PycharmProjects/ugc_analyzer/data/obscene_corpus.txt"
obscene_preprocessed_data_path = "/Users/vladislavasan/PycharmProjects/ugc_analyzer/data/obscene_corpus_preprocessed.txt"
not_obscene_data_path = "/Users/vladislavasan/PycharmProjects/ugc_analyzer/data/2ch_corpus.txt"
not_obscene_preprocessed_data_path = "/Users/vladislavasan/PycharmProjects/ugc_analyzer/data/2ch_corpus_preprocessed.txt"

## Функции для предобработки корпусов

In [ ]:
# использование WhitespaceTokenizer()
# tokenizer.tokenize(text)

# использование word_tokenize
# word_tokenize(text)

Функция лемматизации с возможностью выбора токенайзера

In [3]:
def lemmatization_with_tokenizer(text: str, tokenizer) -> list:
    
    words = tokenizer(text)
    words = [morph.parse(word.lower().strip('\n').strip(punct))[0].normal_form for word in words]

    return words

def lemmatization(words: list) -> list:
    
    words = [morph.parse(word.lower().strip('\n').strip(punct))[0].normal_form for word in words]

    return words

Размер и формат корпуса обсценной лексики позволяет вместить всю необходимую предобработку в одну функцию

In [4]:
def obscene_data_preprocess(data: list):
  
    data = [word.lower().strip('\n').strip(punct) for word in data]

    return data

Размер и формат корпуса с 2ch убивает кернел, если не разделить предобработку на 3 этапа

In [5]:
def not_obscene_data_lemmatization(data: list, tokenizer):

    cleaned_data = []
    for text in tqdm(data):
        text = lemmatization_with_tokenizer(text, tokenizer)
        cleaned_data.append(text)
        
    return cleaned_data

In [6]:
def not_obscene_data_clean1(data: list, obscene_data: list):

    cleaned_data = []
    for text in tqdm(data):
        text = [word for word in text if word not in obscene_data]
        cleaned_data.append(text)
        
    return cleaned_data

In [34]:
def not_obscene_data_clean2(data: list, obscene_data: list):
    
    obscened_words = []

    for word in tqdm(data):
        for obscene_word in obscene_data:
            if obscene_word in word:
                obscened_words.append(word)
    
    obscened_words = set(obscened_words)
    data = set(data)
    data-=obscened_words
    
    return list(data)

## Предбработка корпуса обсценной лексики

In [14]:
with open(obscene_data_path, "r") as obscene_data_file:
    obscene_data = obscene_data_file.readlines()

In [5]:
obscene_data[:20]

['6ЛЯ\n',
 '6ЛЯД\n',
 'FUCK\n',
 'PIZD\n',
 'PIZDA\n',
 'PIZDAH\n',
 'PIZDAKH\n',
 'PIZDAM\n',
 'PIZDAMI\n',
 'PIZDAX\n',
 'PIZDE\n',
 'PIZDOI\n',
 'PIZDOJ\n',
 'PIZDOY\n',
 'PIZDU\n',
 'PIZDY\n',
 'SCHEISE\n',
 'SCHEISSE\n',
 'SHIT\n',
 'SUCK\n']

In [15]:
obscene_data = obscene_data_preprocess(obscene_data)
obscene_data[:10]

['6ля',
 '6ляд',
 'fuck',
 'pizd',
 'pizda',
 'pizdah',
 'pizdakh',
 'pizdam',
 'pizdami',
 'pizdax']

Проверим, что в корпусе нет дубликатов

In [ ]:
len(obscene_data)

7351

In [ ]:
len(set(obscene_data))

7351

## Обработка корпуса с 2ch

In [16]:
with open(not_obscene_data_path, "r") as not_obscene_data_file:
    not_obscene_data = not_obscene_data_file.readlines()

In [17]:
not_obscene_data[:10]

[" Анимублядский WebM-треддля приличных анимублядей и прочих аутистов. Безграмотное быдло с дубляжом, войсовером, порнографией и котиками, советы мерзких мокрописечников, вниманиебляди всех видов и прочее непотребство отправляется в порнотред <ссылка>.Для поиска сoуса видео сохраняем кадр (правый клик по видео) и ищем его на Для воспроизведения WebM с 10-битным цветом нужно установить плагин vlc ( ) и отключить встроенный в браузер плеер (media. webm. enabled=false в firefox).О кодировании WebMДоступные кодеки — VP8 и VP9 для видео, Vorbis и Opus для звука, максимальный размер файла — 10240КБ, всех файлов в посте — около 40МБ. Делать WebM можно научиться в вики треда: Там находится подробная информация о выборе и настройке кодеков на примерах использования консольных утилит ffmpeg, vpxenc и mkvmerge. Неочевидные моменты— libvorbis при указании битрейта (-b:a) работает в режиме CBR (постоянный битрейт), и это портит качество звука; для режима VBR вместо битрейта надо указывать качество 

In [18]:
len(not_obscene_data)

85903

In [19]:
%%time
not_obscene_data = not_obscene_data_lemmatization(not_obscene_data, tokenizer.tokenize)

100%|██████████| 85903/85903 [01:20<00:00, 1062.23it/s]

CPU times: user 1min 15s, sys: 1.53 s, total: 1min 16s
Wall time: 1min 20s


In [20]:
%%time
not_obscene_data = not_obscene_data_clean1(not_obscene_data, obscene_data)

100%|██████████| 85903/85903 [03:57<00:00, 362.05it/s]

CPU times: user 3min 50s, sys: 3.07 s, total: 3min 53s
Wall time: 3min 57s


In [21]:
with open(not_obscene_preprocessed_data_path, "w") as not_obscene_preprocessed_file:
    for word in not_obscene_data:
        not_obscene_preprocessed_file.write(str(word) + "\n")

In [22]:
with open(not_obscene_preprocessed_data_path, "r") as not_obscene_data_file:
    not_obscene_data = not_obscene_data_file.read()

In [24]:
not_obscene_data[:100]

"['анимублядский', 'webm-треддлить', 'приличный', 'анимублядь', 'и', 'прочий', 'аутист', 'безграмотны"

In [25]:
not_obscene_data = not_obscene_data.replace("[", '').replace("]", '').replace("'", '').replace('"', '').replace(',', ' ')

In [26]:
not_obscene_data[:100]

'анимублядский  webm-треддлить  приличный  анимублядь  и  прочий  аутист  безграмотный  быдло  с  дуб'

In [27]:
not_obscene_data = tokenizer.tokenize(not_obscene_data)

In [28]:
not_obscene_data[:10]

['анимублядский',
 'webm-треддлить',
 'приличный',
 'анимублядь',
 'и',
 'прочий',
 'аутист',
 'безграмотный',
 'быдло',
 'с']

In [29]:
len(not_obscene_data)

1786490

In [30]:
len(set(not_obscene_data))

116431

In [31]:
not_obscene_data = list(set(not_obscene_data))

In [35]:
%%time
not_obscene_data_cleaned = not_obscene_data_clean2(not_obscene_data, obscene_data)

100%|██████████| 116431/116431 [00:34<00:00, 3362.12it/s]

CPU times: user 33.6 s, sys: 504 ms, total: 34.1 s
Wall time: 34.7 s


In [36]:
not_obscene_data_cleaned[:10]

['освятить',
 'ординари',
 '>алл',
 'берсеркёр',
 'abyteofpython',
 'дрессировка',
 'вечерник',
 'срочник-корзинка',
 'соизмерение',
 'скинул(']

In [37]:
len(not_obscene_data_cleaned)

107768

In [38]:
not_obscene_data_cleaned = [word.strip(punct) for word in not_obscene_data_cleaned]

In [39]:
not_obscene_data_cleaned[:10]

['освятить',
 'ординари',
 'алл',
 'берсеркёр',
 'abyteofpython',
 'дрессировка',
 'вечерник',
 'срочник-корзинка',
 'соизмерение',
 'скинул']

In [43]:
%%time
with open(not_obscene_preprocessed_data_path, "w") as not_obscene_preprocessed_file:
    for word in tqdm(not_obscene_data_cleaned):
        not_obscene_preprocessed_file.write(word + "\n")

100%|██████████| 107768/107768 [00:00<00:00, 727772.92it/s]

CPU times: user 131 ms, sys: 15.3 ms, total: 146 ms
Wall time: 153 ms


In [44]:
with open(not_obscene_preprocessed_data_path, "r") as not_obscene_data_file:
    not_obscene_data = not_obscene_data_file.readlines()

In [45]:
not_obscene_data[:20]

['освятить\n',
 'ординари\n',
 'алл\n',
 'берсеркёр\n',
 'abyteofpython\n',
 'дрессировка\n',
 'вечерник\n',
 'срочник-корзинка\n',
 'соизмерение\n',
 'скинул\n',
 'бордокультура\n',
 'истоия\n',
 'варащивать\n',
 'пержа\n',
 'потрохов\n',
 'задизайнить\n',
 'посиделки\n',
 'minlen=8\n',
 'дворянин\n',
 'грузчикаэнджа\n']